In [ ]:
%%writefile topsis.py
import sys
import pandas as pd
import numpy as np

def topsis(input_file, weights, impacts, output_file):

    try:
        data = pd.read_csv(input_file)
    except FileNotFoundError:
        print("Error: Input file not found.")
        sys.exit(1)

    if data.shape[1] < 3:
        print("Error: Input file must contain three or more columns.")
        sys.exit(1)

    try:
        matrix = data.iloc[:, 1:].astype(float)
    except ValueError:
        print("Error: From 2nd to last columns must contain numeric values only.")
        sys.exit(1)

    weights = weights.split(',')
    impacts = impacts.split(',')

    if len(weights) != matrix.shape[1]:
        print("Error: Number of weights must match number of criteria.")
        sys.exit(1)

    if len(impacts) != matrix.shape[1]:
        print("Error: Number of impacts must match number of criteria.")
        sys.exit(1)

    for i in impacts:
        if i not in ['+', '-']:
            print("Error: Impacts must be + or -.")
            sys.exit(1)

    weights = np.array(weights, dtype=float)

    norm_matrix = matrix / np.sqrt((matrix ** 2).sum())
    weighted_matrix = norm_matrix * weights

    ideal_best = []
    ideal_worst = []

    for i in range(len(impacts)):
        if impacts[i] == '+':
            ideal_best.append(weighted_matrix.iloc[:, i].max())
            ideal_worst.append(weighted_matrix.iloc[:, i].min())
        else:
            ideal_best.append(weighted_matrix.iloc[:, i].min())
            ideal_worst.append(weighted_matrix.iloc[:, i].max())

    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)

    dist_best = np.sqrt(((weighted_matrix - ideal_best) ** 2).sum(axis=1))
    dist_worst = np.sqrt(((weighted_matrix - ideal_worst) ** 2).sum(axis=1))

    score = dist_worst / (dist_best + dist_worst)

    data['Topsis Score'] = score
    data['Rank'] = data['Topsis Score'].rank(ascending=False).astype(int)

    data.to_csv(output_file, index=False)
    print("Topsis completed :)")

if __name__ == "__main__":

    if len(sys.argv) != 5:
        print("Usage:")
        print("python topsis.py <InputDataFile> <Weights> <Impacts> <OutputResultFileName>")
        sys.exit(1)

    _, input_file, weights, impacts, output_file = sys.argv
    topsis(input_file, weights, impacts, output_file)


Writing topsis.py


In [2]:
import pandas as pd

df = pd.read_excel("data.xlsx")
df.to_csv("data.csv", index=False)

print("Converted to data.csv")


Converted to data.csv


In [3]:
!ls

data.csv  data.xlsx  sample_data  topsis.py


In [7]:
!python topsis.py data.csv "1,1,1,1,1" "+,+,-,+,-" output.csv

Topsis completed :)
